In [1]:
# Install necessary libraries
!pip install keras keras-preprocessing tensorflow spacy pydantic
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from google.colab import files
uploaded = files.upload()

# Import libraries

In [2]:
import spacy
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from random import randint
from pickle import dump, load

In [3]:
# Read file
def read_file(filepath):
    with open(filepath, 'r') as f:
        return f.read()

# Tokenize and clean text
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])
nlp.max_length = 1198623

def separate_punc(text):
    return [token.text.lower() for token in nlp(text) if token.text.isalpha()]

text = read_file('Harry_Potter_1_Chapter1.txt')
tokens = separate_punc(text)

/usr/local/lib/python3.11/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [4]:
# Create sequences
seq_len = 25
sequences = [
    tokens[i-seq_len:i]
    for i in range(seq_len, len(tokens))
]

# Tokenize sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
encoded_sequences = np.array(tokenizer.texts_to_sequences(sequences))

# Prepare input and target
X, y = encoded_sequences[:, :-1], encoded_sequences[:, -1]
y = to_categorical(y, num_classes=len(tokenizer.word_counts) + 1)


In [5]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

def create_model(vocab_size, seq_len, lstm_units):
    model = Sequential([
        Embedding(vocab_size, 50, input_length=seq_len),
        LSTM(lstm_units, return_sequences=True),
        LSTM(lstm_units),
        Dense(256, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

vocab_size = len(tokenizer.word_index) + 1

# Small Model
small_model = create_model(vocab_size, seq_len, lstm_units=50)

# Big Model
big_model = create_model(vocab_size, seq_len, lstm_units=100)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
# Small Model
small_model.fit(X, y, batch_size=128, epochs=20)

# Save the small model
small_model.save('harry_potter_small_model.h5')

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.0274 - loss: 6.8791
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.0415 - loss: 6.0097
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.0415 - loss: 5.9436
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.0440 - loss: 5.9617
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.0482 - loss: 5.9070
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.0350 - loss: 5.8867
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - accuracy: 0.0390 - loss: 5.8902
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.0436 - loss: 5.7748
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.0378 - loss: 5.7598
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.0430 - loss: 5.6628
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.0482 - loss: 5.6314
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy

In [7]:
# Big Model
big_model.fit(X, y, batch_size=128, epochs=40)

# Save the big model
big_model.save('harry_potter_big_model.h5')

Epoch 1/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 118ms/step - accuracy: 0.0395 - loss: 6.7864
Epoch 2/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 155ms/step - accuracy: 0.0370 - loss: 6.0267
Epoch 3/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 104ms/step - accuracy: 0.0439 - loss: 5.9236
Epoch 4/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.0393 - loss: 5.9630
Epoch 5/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 117ms/step - accuracy: 0.0483 - loss: 5.8778
Epoch 6/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.0474 - loss: 5.8156
Epoch 7/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 146ms/step - accuracy: 0.0414 - loss: 5.7363
Epoch 8/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 105ms/step - accuracy: 0.0422 - loss: 5.6604
Epoch 9/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 152ms/step - accuracy: 0.0476 - loss: 5.5621
Epoch 10/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 103ms/step - accuracy: 0.0492 - loss: 5.5066
Epoch 11/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 221ms/step - accuracy: 0.0558 - loss: 5.3622
Epoch 12/40
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step

In [9]:
from keras.models import load_model

# Load the models
model1 = load_model('/content/harry_potter_small_model.h5')
model2 = load_model('/content/harry_potter_big_model.h5')

# Evaluate on test data (X_test, y_test)
model1_results = model1.evaluate(X, y, verbose=0)
model2_results = model2.evaluate(X, y, verbose=0)

print(f"Model 1 Results: Loss = {model1_results[0]}, Accuracy = {model1_results[1]}")
print(f"Model 2 Results: Loss = {model2_results[0]}, Accuracy = {model2_results[1]}")

Model 1 Results: Loss = 4.9745259284973145, Accuracy = 0.06706244498491287
Model 2 Results: Loss = 3.785808563232422, Accuracy = 0.1312664896249771


In [10]:
from keras.models import load_model
from random import randint

def generate_text(model, tokenizer, seq_len, seed_text, num_words):
    input_text = seed_text
    output_text = []

    for _ in range(num_words):
        encoded = tokenizer.texts_to_sequences([input_text])[0]
        padded = pad_sequences([encoded], maxlen=seq_len, truncating='pre')
        pred_index = np.argmax(model.predict(padded), axis=1)[0]
        pred_word = tokenizer.index_word.get(pred_index, '')
        input_text += ' ' + pred_word
        output_text.append(pred_word)

    return ' '.join(output_text)


compare the smaler lstm_unit vs bigger lstm_unit

In [12]:
# Seed Text
seed_index = randint(0, len(sequences))
seed_text = ' '.join(sequences[seed_index])

# Generate text
small_model_text = generate_text(small_model, tokenizer, seq_len, seed_text, 500)
big_model_text = generate_text(big_model, tokenizer, seq_len, seed_text, 500)

# Save the generated chapters
with open('harry_potter_small_model.txt', 'w', encoding='utf-8') as f:
    f.write(small_model_text)

with open('harry_potter_big_model.txt', 'w', encoding='utf-8') as f:
    f.write(big_model_text)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

In [13]:
with open('harry_potter_small_model.txt', 'r', encoding='utf-8') as f:
    small_model_chapter = f.read()

with open('harry_potter_big_model.txt', 'r', encoding='utf-8') as f:
    big_model_chapter = f.read()


In [ ]:
from keras.layers import Dropout

# adding dropout layer
def create_model_adding_drop_out_layer(vocab_size, seq_len, lstm_units=150):
    model = Sequential([
        Embedding(vocab_size, 100, input_length=seq_len),
        LSTM(lstm_units, return_sequences=True),
        Dropout(0.2),
        LSTM(lstm_units),
        Dropout(0.2),
        Dense(lstm_units, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
model3 = create_model_adding_drop_out_layer(vocab_size, seq_len, lstm_units = 100)

In [ ]:
# Model3
model3.fit(X, y, batch_size=128, epochs=40)

# Save the model3
model3.save('harry_potter_model3.h5')

increase lstm_units

In [ ]:
model4 = create_model_adding_drop_out_layer(vocab_size, seq_len, lstm_units = 200)

In [ ]:
# Model3
model4.fit(X, y, batch_size=128, epochs=40)

# Save the model3
model4.save('harry_potter_model3.h5')

In [ ]:
# Generate text
model3 = generate_text(small_model, tokenizer, seq_len, seed_text, 500)
model4 = generate_text(big_model, tokenizer, seq_len, seed_text, 500)

# Save the generated chapters
with open('harry_potter_model3.txt', 'w', encoding='utf-8') as f:
    f.write(small_model_text)

with open('harry_potter_model4.txt', 'w', encoding='utf-8') as f:
    f.write(big_model_text)


Let's modify the model to include dropout layers and increase the number of epochs from 40 to 100:

In [ ]:
# Model3
model4.fit(X, y, batch_size=128, epochs=1003)

# Save the model3
model4.save('harry_potter_model3.h5')